In [9]:
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time 
import random

chromedriver_path = r'C:\Users\luisf\Desktop\MDS_thesis\scraper_gn\chromedriver-win64\chromedriver.exe'
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

In [10]:
search_query = "amlo dice"
start_date = "2018-12-03"
end_date = "2024-09-22"
max_pages = 5
limit=100 
count=0
current_page=1
url = f'https://www.google.com/search?q={search_query}+after:{start_date}+before:{end_date}&tbm=nws'
driver.get(url)


In [11]:
#Second iteration with pagination... fully functional. 
news_data = []

while current_page <= max_pages and count < limit:
    print(f"Scraping page {current_page}")
    
    try:
        # Wait for the news results to load
        WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.SoaBEf'))
        )
    except Exception as e:
        print(f"Timeout while waiting for news articles on page {current_page}: {e}")
        # Optional: Take a screenshot for debugging
        driver.save_screenshot(f'timeout_page_{current_page}.png')
        break  # Exit if articles do not load in time

    # Select all news article blocks by targeting the SoaBEf class
    news_results = driver.find_elements(By.CSS_SELECTOR, 'div.SoaBEf')
    
    if not news_results:
        print(f"No articles found on page {current_page}.")
        # Optional: Take a screenshot to inspect the page
        driver.save_screenshot(f'no_articles_page_{current_page}.png')
        break  # Exit if no articles are found

    for news_div in news_results:
        if count >= limit:
            print(f"Reached the limit of {limit} articles.")
            break  # Stop if we've reached the limit

        try:
            # Extract the link to the article
            news_link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')

            # Extract the title of the article
            news_title = news_div.find_element(By.CSS_SELECTOR, 'a div[role="heading"]').text

            # Extract the source domain
            news_domain = news_div.find_element(By.CSS_SELECTOR, 'div').text

            # Extract the snippet description and the publication date
            news_description = news_div.find_element(By.CSS_SELECTOR, 'a div[role="heading"] + div').text
            news_date = news_div.find_element(By.CSS_SELECTOR, 'span').text

            # Append the collected information to the list
            news_data.append([news_link, news_domain, news_title, news_description, news_date])
            count += 1  # Increment the count of news articles

        except Exception as e:
            print(f"Error scraping an article on page {current_page}: {e}")
            # Optional: Take a screenshot for debugging
            driver.save_screenshot(f'error_article_page_{current_page}.png')
            continue  # Skip to the next article

    # Check if we've reached the limit after scraping the current page
    if count >= limit:
        print(f"Reached the limit of {limit} articles.")
        break

    # Attempt to locate the "Next" button and click it using the general selector
    try:
        # Find the "Next" button using its general ID selector
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#pnnext'))
        )
        print("Found 'Next' button using the '#pnnext' selector.")
        
        # Scroll to the "Next" button to ensure it's in view
        driver.execute_script("arguments[0].scrollIntoView();", next_button)
        
        # Introduce a random sleep to mimic human behavior
        time.sleep(random.uniform(2, 5))
        
        # Capture the current URL before clicking "Next"
        current_url = driver.current_url
        print(f"Current URL before clicking 'Next': {current_url}")
        
        # Click the "Next" button
        next_button.click()
        print(f"Clicked 'Next' button on page {current_page}.")
        
        # Increment the current page counter
        current_page += 1
        
        # Optional: Add a short sleep to wait for the next page to load
        time.sleep(random.uniform(2, 4))
        
        # Capture the new URL after clicking "Next"
        new_url = driver.current_url
        print(f"Navigated to new URL: {new_url}")
        
        # Verify that the URL has changed to prevent looping
        if new_url == current_url:
            print("URL did not change after clicking 'Next'. Possible loop detected.")
            break  # Exit the loop to prevent infinite cycling
        else:
            print(f"Successfully navigated to page {current_page}.")

    except Exception as e:
        print(f"No more pages available or error clicking 'Next' on page {current_page}: {e}")
        # Optional: Take a screenshot to inspect the state when the error occurred
        driver.save_screenshot(f'next_button_error_page_{current_page}.png')
        break  # Exit the loop if there's no next button or an error occurs

Scraping page 1
Found 'Next' button using the '#pnnext' selector.
Current URL before clicking 'Next': https://www.google.com/search?q=amlo+dice+after:2018-12-03+before:2024-09-22&tbm=nws
Clicked 'Next' button on page 1.
Navigated to new URL: https://www.google.com/search?q=amlo+dice+after:2018-12-03+before:2024-09-22&sca_esv=4997d9601951f80e&sca_upv=1&tbm=nws&ei=Gfn8ZuTjCeWMxc8PuNPtgQI&start=10&sa=N&ved=2ahUKEwik1MD_mO-IAxVlRvEDHbhpOyAQ8NMDegQIAhAY&biw=929&bih=865&dpr=1
Successfully navigated to page 2.
Scraping page 2
Found 'Next' button using the '#pnnext' selector.
Current URL before clicking 'Next': https://www.google.com/search?q=amlo+dice+after:2018-12-03+before:2024-09-22&sca_esv=4997d9601951f80e&sca_upv=1&tbm=nws&ei=Gfn8ZuTjCeWMxc8PuNPtgQI&start=10&sa=N&ved=2ahUKEwik1MD_mO-IAxVlRvEDHbhpOyAQ8NMDegQIAhAY&biw=929&bih=865&dpr=1
Clicked 'Next' button on page 2.
Navigated to new URL: https://www.google.com/search?q=amlo+dice+after:2018-12-03+before:2024-09-22&sca_esv=4997d9601951f80e

In [13]:
print(news_data)
print(len(news_data))

[['https://efe.com/mundo/2024-06-17/mexico-amlo-se-dice-muy-satisfecho-por-eleccion-de-sheinbaum/', 'Agencia EFE\nMéxico: AMLO se dice "muy satisfecho" por elección de Sheinbaum\nLa gira de Sheinbaum y López Obrador por México continúa. El gobernante mexicano aseveró que “es algo inédito” la gira que comenzaron ambos políticos el viernes...\n.\n17 jun 2024', 'México: AMLO se dice "muy satisfecho" por elección de Sheinbaum', 'La gira de Sheinbaum y López Obrador por México continúa. El gobernante mexicano aseveró que “es algo inédito” la gira que comenzaron ambos políticos el viernes...', 'Agencia EFE'], ['https://www.bbc.com/mundo/noticias-internacional-53344257', 'BBC\nDiscurso de AMLO: el agradecimiento a Trump y otros momentos llamativos del encuentro de ambos presidentes en la Casa Blanca\nEl presidente de México, Andrés Manuel López Obrador, miró a los ojos a su par de Estados Unidos, Donald Trump, y le dijo: "Fallaron los pronósticos,...\n.\n9 jul 2020', 'Discurso de AMLO: el agr

In [36]:
df = pd.DataFrame(news_data, columns=["Link", "Domain", "Title", "Description", "Date"])
print(df)
print(len(df))

                                              Link Domain  \
0  https://www.bbc.com/mundo/articles/cv22e6g3x59o    BBC   

                                               Title  \
0  AMLO: los 4 pilares que explican su alta popul...   

                                         Description         Date  
0  A cuatro meses del fin de su mandato, el presi...  29 may 2024  
1


In [ ]:

# Save the results to a CSV file
csv_file_path = "news_results.csv"
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the header row
    writer.writerow(["Link", "Domain", "Title", "Description", "Date"])
    # Write the news data
    writer.writerows(news_data)

driver.quit()

print(f"Results saved to {csv_file_path}")

Complete code for scraping and iterating over pages 

In [12]:
import csv
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from urllib.parse import quote
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import random

# -----------------------------
# Configure Chrome Options
# -----------------------------
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-dev-shm-usage")
user_agent = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/112.0.0.0 Safari/537.36"
)
chrome_options.add_argument(f"user-agent={user_agent}")

# -----------------------------
# Initialize WebDriver
# -----------------------------
chromedriver_path = r'C:\Users\luisf\Desktop\MDS_thesis\scraper_gn\chromedriver-win64\chromedriver.exe'
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# -----------------------------
# Define Search Parameters
# -----------------------------
search_query = "amlo dice"
start_date = "2018-12-03"
end_date = "2024-09-22"
max_pages = 3  # Set the number of pages to scrape
limit = 100  # Set the limit for the number of articles
count = 0
current_page = 1

# Encode the search query for URL
encoded_query = quote(search_query)
url = f'https://www.google.com/search?q={encoded_query}+after:{start_date}+before:{end_date}&tbm=nws'


In [11]:

# Initialize a list to hold the results
news_data = []

try:
    driver.get(url)
    
    while current_page <= max_pages and count < limit:
        print(f"Scraping page {current_page}")
        
        try:
            # Wait for the news results to load
            WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.SoaBEf'))
            )
        except TimeoutException:
            print(f"Timeout while waiting for news articles on page {current_page}.")
            break  # Exit if articles do not load in time
        
        # Select all news article blocks
        news_results = driver.find_elements(By.CSS_SELECTOR, 'div.SoaBEf')
        
        for news_div in news_results:
            if count >= limit:
                break  # Stop if we've reached the limit

            try:
                # Extract the link to the article
                news_link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')

                # Extract the title of the article
                news_title = news_div.find_element(By.CSS_SELECTOR, 'div[role="heading"]').text

                # Extract the source domain
                try:
                    news_domain = news_div.find_element(By.CSS_SELECTOR, 'div.XTjFC').text
                except NoSuchElementException:
                    news_domain = "Domain not found"

                # Extract the description
                try:
                    news_description = news_div.find_element(By.CSS_SELECTOR, 'div.Y3v8qd').text
                except NoSuchElementException:
                    news_description = "Description not found"

                # Extract the date
                try:
                    news_date = news_div.find_element(By.CSS_SELECTOR, 'span.WG9SHc').text
                except NoSuchElementException:
                    news_date = "Date not found"

                # Append the collected information to the list
                news_data.append([news_link, news_domain, news_title, news_description, news_date])
                count += 1  # Increment the count of news articles

            except Exception as e:
                print(f"Error scraping an article on page {current_page}: {e}")
                continue  # Skip to the next article
        
        if count >= limit:
            print(f"Reached the limit of {limit} articles.")
            break

        # Attempt to click the "Next" button to go to the next page
        try:
            # Locate the "Next" button by its aria-label
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//a[@aria-label="Next"]'))
            )
            
            # Scroll to the "Next" button to ensure it's in view
            driver.execute_script("arguments[0].scrollIntoView();", next_button)
            
            # Introduce a random sleep to mimic human behavior
            time.sleep(random.uniform(2, 5))
            
            # Click the "Next" button
            next_button.click()
            
            # Increment the current page counter
            current_page += 1
            
        except TimeoutException:
            print("No more pages available or 'Next' button not found.")
            break  # Exit the loop if there's no next button
        except ElementClickInterceptedException as e:
            print(f"Could not click the 'Next' button on page {current_page}: {e}")
            break  # Exit the loop if clicking fails
        except Exception as e:
            print(f"Unexpected error while navigating to next page: {e}")
            break  # Exit the loop on any other unexpected errors

        # Optional: Add a short sleep before scraping the next page
        time.sleep(random.uniform(1, 3))
    
    
    # -----------------------------
    # Save the Data to a CSV File
    # -----------------------------
    df = pd.DataFrame(news_data, columns=["Link", "Domain", "Title", "Description", "Date"])
    df.to_csv('news_data.csv', index=False, encoding='utf-8')
    print(f"Scraping completed. {count} articles saved to 'news_data.csv'.")
    
except Exception as e:
    print(f"An unexpected error occurred: {e}")

finally:
    driver.quit()


Scraping page 1
Timeout while waiting for news articles on page 1.
Scraping completed. 0 articles saved to 'news_data.csv'.


In [48]:
print(df)

Empty DataFrame
Columns: [Link, Domain, Title, Description, Date]
Index: []
